## Introduction
Erwin de Leon and Joeseph Schilling introduce their April 2017 Urban Instute research report ["Urban Blight and Public Health: Addressing the Impact of Substandard Housing, Abandoned Buildings, and Vacant Lots"](https://www.urban.org/research/publication/urban-blight-and-public-health) with the following statement: "We spend more than 2/3rds of our time where we live; thus, housing and neighborhood conditions affect our individual and family's well-being". They also discuss the impact of poor economic conditions that result in "increasing inventories of vacant homes and abandoned buildings". For example, the authors of this report cite Detroit's Blight Removal Task Force's 2014 "strategic plan to address more than 80,000 derelict structures and vacant lots".

This Jupyter notebook illustrates the application of the Python programming language to address three questions regarding the [City of Detroit's Blight Violation Notices (BVN) public domain dataset](https://data.detroitmi.gov/Property-Parcels/Blight-Violations/ti6p-wcg4):  
1. Is the number of blight violations per month increasing, decreasing, or staying constant over time?  
2. Is the number of blight violations in collection increasing, decreasing, or staying constant over time?  
3. For 2017, are blight violation notices clustered as a function of latitute and longitude?  

## Import Dependencies

In [407]:
import numpy as np
import pandas as pd
from csv import DictReader
import os
import urllib
import re
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from itertools import zip_longest

%load_ext pycodestyle_magic
%matplotlib inline

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


## Prepare Data  
-[Download *.csv from URL](https://stackoverflow.com/questions/41992223/download-csv-from-web-service-with-python-3)  
-[Verify PEP8 in IPython notebook](https://stackoverflow.com/questions/26126853/verifying-pep8-in-ipython-notebook-code)  
-[Tidy Data](https://www.jstatsoft.org/article/view/v059i10)  
-[How to read file n lines at a time in Python](https://stackoverflow.com/questions/5832856/how-to-read-file-n-lines-at-a-time-in-python)  
-[Delete last item in a list](https://stackoverflow.com/questions/18169965/how-to-delete-last-item-in-list)  
-[Remove comma from the end of a line](https://stackoverflow.com/questions/2774558/how-do-i-strip-the-comma-from-the-end-of-a-string-in-python)  
-[Appending row to Pandas DataFrame can be slow](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html)  
-[DictReader parse string](https://stackoverflow.com/questions/31658115/python-csv-dictreader-parse-string)  
-[Pandas replace nan with blank empty string](https://stackoverflow.com/questions/26837998/pandas-replace-nan-with-blank-empty-string)  


In [241]:
#%%pycodestyle
csv_url = "https://data.detroitmi.gov/api/views/ti6p-wcg4" +\
          "/rows.csv?accessType=DOWNLOAD"

data_dir = './Data'
csv_file = os.path.join(data_dir, 'rows.csv')

if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    urllib.request.urlretrieve(csv_url, csv_file)

In [417]:
#%%pycodestyle
def format_columns(h_file):
    """
    Formats the columns of the City of Detroit Blight Violation Notices
    (BVN) public domain dataset

    INPUT:
        h_file: BVN *.csv file handle

    OUTPUT:
        columns: formatted list of BVN *.csv file columns
    """
    columns = h_file.readline().strip()

    columns = [elem.lower() for elem in columns.split(',')]

    columns = [re.sub("[\\s\\(\\)-]", '', elem) for elem in columns]

    del columns[-1]

    return columns


def parse_string(string_value):
    """
    If the input string value is empty, returns "NULL". Otherwise,
    returns the input string value

    INPUT:
        string_value: String value that may be empty

    RETURNS:
        formatted_string_value: "NULL" if the input string value
            is empty. Otherwise, is set to the input string value
    """
    if string_value == '':
        return "NULL"
    else:
        return string_value


def parse_int(int_string):
    """Parses an integer value stored in a string

    INPUT:
        int_string: String that stores an integer value

    OUTPUT:
        int_value: Integer value stored in a string"""
    try:
        int_value = int(int_string)
    except ValueError:
        int_value = np.nan

    return int_value


def parse_float(float_string):
    """Parses a floating point value stored in a string

    INPUT:
        float_string: String that stores a floating point value

    OUTPUT:
        float_value: Floating point value stored in a string"""
    try:
        float_value = float(float_string)
    except ValueError:
        float_value = np.nan

    return float_value


class RowFormatter(object):
    """
    Defines the format of a City of Detroit Blight Violation Notices
    dataset row
    """

    def __init__(self):
        """BVN dataset row formatter class object constructor

        INPUT:
            self: RowFormatter class object reference

        OUTPUT:
            self: RowFormatter class object reference
        """
        self.conversion =\
            {'ticketid': lambda x: parse_int(x),
             'ticketnumber': lambda x: parse_string(x),
             'agencyname': lambda x: parse_string(x),
             'inspectorname': lambda x: parse_string(x),
             'violatorname': lambda x: parse_string(x),
             'violatorid': lambda x: parse_int(x),
             'violationstreetnumber':
             lambda x: parse_int(x),
             'violationstreetname':
             lambda x: parse_string(x),
             'violationzipcode':
             lambda x: parse_int(x),
             'mailingaddressstreetnumber':
             lambda x: parse_int(x),
             'mailingaddressstreetname':
             lambda x: parse_string(x),
             'mailingaddresscity':
             lambda x: parse_string(x),
             'mailingaddressstate':
             lambda x: parse_string(x),
             'mailingaddresszipcode':
             lambda x: parse_int(x),
             'mailingaddressnonusacode':
             lambda x: parse_int(x),
             'mailingaddresscountry':
             lambda x: parse_string(x),
             'violationdate': lambda x: parse_string(x),
             'ticketissuedtime': lambda x: parse_string(x),
             'hearingdate': lambda x: parse_string(x),
             'hearingtime': lambda x: parse_string(x),
             'violationcode': lambda x: parse_string(x),
             'violationdescription':
             lambda x: parse_string(x),
             'disposition': lambda x: parse_string(x),
             'fineamount': lambda x: parse_float(x),
             'adminfee': lambda x: parse_float(x),
             'statefee': lambda x: parse_float(x),
             'latefee': lambda x: parse_float(x),
             'discountamount': lambda x: parse_float(x),
             'cleanupcost': lambda x: parse_float(x),
             'judgmentamounttotaldue':
             lambda x: parse_float(x),
             'paymentamountsumofallpayments':
             lambda x: parse_float(x),
             'balancedue': lambda x: parse_float(x),
             'paymentdatemostrecent':
             lambda x: parse_string(x),
             'paymentstatus':
             lambda x: parse_string(x),
             'collectionstatus': lambda x: parse_string(x),
             'violationaddress': lambda x: parse_string(x),
             'violationparcelid': lambda x: parse_string(x),
             'violationlatitude': lambda x: parse_float(x),
             'violationlongitude': lambda x: parse_float(x)}

    def format_row(self,
                   row):
        """
        Returns an ordered dictonary that stores a formatted
        BVN datset row

        INPUT:
            self: RowFormatter class reference

            row: String that stores a BVN dataset row

        OUTPUT:
            row_dict: Ordered dictonary that stores a formatted
                      BVN datset row
        """
        reader =\
            DictReader([row],
                       fieldnames=self.conversion.keys())

        try:
            row_dict = reader.__next__()

            for key in list(row_dict.keys()):
                row_dict[key] = self.conversion[key](row_dict[key])
        except TypeError:
            row_dict = None

        return row_dict


def initialize_bvn_df(bvn_csv_file):
    """
    Formats City of Detroit Blight Violation Notices (BVN) and
    stores them in a Pandas DataFrame

    INPUT:
        bvn_csv_file: Full path to a City of Detroit (BVN)
                      *.csv file

    OUTPUT:
        bvn_df: Pandas DataFrame that stores City of Detroit
                Blight Violation Notices
    """
    with open(bvn_csv_file) as h_file:
        formatter_obj = RowFormatter()

        columns = format_columns(h_file)

        pattern_obj =\
            re.compile('^[a-z"\\s]*\\([0-9\\.-]+, [0-9\\.-]+\\)"')

        rows = []
        number_records = 0
        malformed_records = 0

        for cur_line in tqdm(h_file):
            cur_line = cur_line.strip()

            if pattern_obj.match(cur_line) is None:
                number_records += 1

                cur_line = re.sub('"location', '', cur_line)
                cur_line = cur_line.rstrip(',')

                row_dict = formatter_obj.format_row(cur_line)

                if row_dict is None:
                    malformed_records += 1
                else:
                    rows.append(list(row_dict.values()))

    print("Dropped %.2f %% of BVN's" %
          (100 * malformed_records / number_records))

    bvn_df = pd.DataFrame(rows, columns=columns)

    return bvn_df.replace("NULL", None, regex=True)

## Load BVN dataset
- Results suggest that less than 1% of the BVN records were dropped
    - A record is dropped if a formatting error occurs  

In [432]:
df = initialize_bvn_df(csv_file)

784384it [00:13, 57664.58it/s]


Dropped 0.24 % of BVN's


In [420]:
print("(# of rows, # of columns): (%d, %d)" % (df.shape[0], df.shape[1]))

(# of rows, # of columns): (391516, 39)


In [429]:
missing_rows = df.isnull().sum(axis=0)
missing_rows = missing_rows[missing_rows > 0]
missing_rows = missing_rows.sort_values(ascending=False)
missing_rows

mailingaddressnonusacode         391471
cleanupcost                      386834
violationzipcode                 329556
paymentamountsumofallpayments    188842
mailingaddressstreetnumber         4874
mailingaddresszipcode              3186
fineamount                            2
dtype: int64

In [431]:
percent_missing_rows = missing_rows * (100 / df.shape[0])
percent_missing_rows

mailingaddressnonusacode         99.988506
cleanupcost                      98.804136
violationzipcode                 84.174338
paymentamountsumofallpayments    48.233533
mailingaddressstreetnumber        1.244904
mailingaddresszipcode             0.813760
fineamount                        0.000511
dtype: float64

## Remove columns with a large amount of missing data  
-[Yiran Dong and Chao-Ying Joanne Peng "Principled missing data methods for researchers"](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3701793/)

In [436]:
columns_to_drop = list(percent_missing_rows[percent_missing_rows > 10].index)
print(columns_to_drop)
df = df.filter([elem for elem in df.columns if elem not in columns_to_drop])

['mailingaddressnonusacode', 'cleanupcost', 'violationzipcode', 'paymentamountsumofallpayments']


In [448]:
missing_columns = df.isnull().sum(axis=1)
missing_columns.shape

(391516,)